In [122]:
import os

os.environ['KAGGLE_USERNAME'] = 'yannspf'
os.environ['KAGGLE_KEY'] = '47e6ebd956faccc933d2bba9d554bbbe'
try:
    import kaggle
except ImportError:
    os.system('pip install kaggle')

In [123]:
import kagglehub

# Billboard Hot 100 dataset
path1 = kagglehub.dataset_download("thedevastator/billboard-hot-100-audio-features")
print("Path to Billboard dataset:", path1)

# Spotify Top Songs dataset
path2 = kagglehub.dataset_download("asaniczka/top-spotify-songs-in-73-countries-daily-updated")
print("Path to Spotify dataset:", path2)

Path to Billboard dataset: /Users/yann/.cache/kagglehub/datasets/thedevastator/billboard-hot-100-audio-features/versions/1
Path to Spotify dataset: /Users/yann/.cache/kagglehub/datasets/asaniczka/top-spotify-songs-in-73-countries-daily-updated/versions/579


In [124]:
import pandas as pd

# Load and preview Billboard datasets
audio_features_df = pd.read_csv(os.path.join(path1, "Hot 100 Audio Features.csv"))
hot_stuff_df = pd.read_csv(os.path.join(path1, "Hot Stuff.csv"))

# Load and preview Spotify dataset
spotify_df = pd.read_csv(os.path.join(path2, "universal_top_spotify_songs.csv"))

# Print first 5 rows of each dataset
print(" First 5 rows of 'Hot 100 Audio Features.csv':")
print(audio_features_df['spotify_genre'].head())


 First 5 rows of 'Hot 100 Audio Features.csv':
0                                                   []
1                                          ['novelty']
2    ['adult standards', 'brill building pop', 'eas...
3     ['rock-and-roll', 'space age pop', 'surf music']
4                ['dance pop', 'pop', 'post-teen pop']
Name: spotify_genre, dtype: object


In [125]:
import ast

# Drop NaN values to avoid errors
genre_series = audio_features_df['spotify_genre'].dropna()

# Parse genre strings to lists or empty list if NaN
audio_features_df['parsed_genres'] = audio_features_df['spotify_genre'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

# Drop rows where the parsed list is empty
audio_features_df = audio_features_df[audio_features_df['parsed_genres'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

# (Optional) Reset index
audio_features_df.reset_index(drop=True, inplace=True)

# Convert stringified lists to real Python lists using ast.literal_eval
parsed_genres = genre_series.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Flatten all lists into one
all_genres = [genre for sublist in parsed_genres for genre in sublist]

# Get unique genres
unique_genres = sorted(set(all_genres))

# Display results
print(f"🎵 Total unique genres: {len(unique_genres)}")
print(unique_genres)


🎵 Total unique genres: 1145
['21st century classical', 'a cappella', 'acid house', 'acid jazz', 'acoustic blues', 'acoustic pop', 'acoustic punk', 'adult standards', 'afro house', 'afrobeat', 'afropop', 'alabama indie', 'alabama metal', 'alabama rap', 'alaska indie', 'alberta country', 'album rock', 'albuquerque indie', 'alt z', 'alt-idol', 'alternative americana', 'alternative country', 'alternative dance', 'alternative emo', 'alternative hip hop', 'alternative metal', 'alternative pop', 'alternative pop rock', 'alternative r&b', 'alternative rock', 'alternative roots rock', 'ambient folk', 'ambient psychill', 'american folk revival', 'american post-rock', 'anarcho-punk', 'anime', 'ann arbor indie', 'anthem emo', 'anthem worship', 'anti-folk', 'antiviral pop', 'appalachian folk', 'argentine rock', 'arkansas country', 'arkansas hip hop', 'art pop', 'art rock', 'asmr', 'athens indie', 'atl hip hop', 'atl trap', 'audiophile vocal', 'aussietronica', 'austindie', 'australian alternative ro

In [126]:


genre_map = {
    'novelty': 'Pop',
    'idol': 'Pop',
    'boy band': 'Pop',
    'broadway': 'Pop',
    'classic girl group': 'Pop',
    'calming instrumental': 'Easy Listening',
    'neo mellow': 'Pop',
    'urban contemporary': 'R&B',

    'motown': 'R&B',
    'deep motown': 'R&B',
    'rare groove': 'R&B',
    'new jack swing': 'R&B',

    'comic': 'Comedy',
    'comedy': 'Comedy',
    'parody': 'Comedy',

    'hollywood': 'Soundtrack',
    'movie tunes': 'Soundtrack',
    'show tunes': 'Soundtrack',
    'christmas instrumental': 'Holiday',

    'nashville sound': 'Country',
    'norteno': 'Regional Mexican',
    'beach music': 'Soul',

    'latin': 'Latin',
    'tropical': 'Latin',
    'brooklyn drill': 'Hip Hop',

    'merseybeat': 'Rock',
    'british invasion': 'Rock',
    'adult standards': 'Easy Listening',
    'easy listening': 'Easy Listening',
    'deep adult standards': 'Easy Listening',
    'mellow gold': 'Easy Listening',
    'lounge': 'Easy Listening',
    
    'freestyle': 'Dance',
    'disco': 'Dance',
    'hi-nrg': 'Dance',
    'miami bass': 'Dance',

    'quiet storm': 'R&B',
    'doo-wop': 'R&B',
    'vocal harmony group': 'R&B',

    'surf music': 'Rock',
    'cowboy western': 'Country',
    'bakersfield sound': 'Country',

    'ska': 'Ska',
    'ska punk': 'Ska',
    'ska revival': 'Ska',
    'traditional ska': 'Ska',
    'german ska': 'Ska',
    'pop': 'Pop',
    'rock': 'Rock',
    'hip hop': 'Hip Hop',
    'rap': 'Hip Hop',
    'r&b': 'R&B',
    'edm': 'Electronic',
    'electro': 'Electronic',
    'house': 'Electronic',
    'dance': 'Dance',
    'metal': 'Metal',
    'jazz': 'Jazz',
    'classical': 'Classical',
    'country': 'Country',
    'folk': 'Folk',
    'reggae': 'Reggae',
    'trap': 'Trap',
    'blues': 'Blues',
    'indie': 'Indie',
    'soul': 'Soul',
    'funk': 'Funk'
}
# Function to assign a broad genre
def map_to_broad_genre(genre_list, genre_map):
    if not isinstance(genre_list, list):
        return ['Other']
    
    matched = []
    for genre in genre_list:
        for key in genre_map:
            if key in genre.lower():
                matched.append(genre_map[key])
                break  # Avoid multiple matches per input genre

    return matched if matched else ['Other']


# Parse the spotify_genre column into actual lists
audio_features_df['parsed_genres'] = audio_features_df['spotify_genre'].dropna().apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Assign each row a broad genre
audio_features_df['broad_genre'] = audio_features_df['parsed_genres'].apply(
    lambda genres: map_to_broad_genre(genres, genre_map))

# Show genre distribution
genre_counts = audio_features_df['broad_genre'].value_counts()
print("🎼 Broad genre distribution:")
print(genre_counts)

#  Identify and analyze the "Other" genres
other_rows = audio_features_df[audio_features_df['broad_genre'] == 'Other']
other_genres = [
    genre for sublist in other_rows['parsed_genres'] if isinstance(sublist, list) for genre in sublist
]
other_genre_counts = Counter(other_genres)

# Save unmatched genres to CSV
other_genre_df = pd.DataFrame(
    other_genre_counts.items(), columns=['genre', 'count']
).sort_values(by='count', ascending=False)

other_genre_df.to_csv("unmatched_other_genres.csv", index=False)
audio_features_df.to_csv("broad_genres.csv", index=False)



🎼 Broad genre distribution:
broad_genre
[Pop]                                                           807
[Pop, Pop]                                                      737
[Pop, Pop, Pop]                                                 557
[Rock]                                                          404
[Country, Country, Country, Rock]                               399
                                                               ... 
[Rock, Pop, Rock, Pop, Pop, Rock]                                 1
[Pop, Electronic, Electronic, Pop, Electronic, Latin]             1
[Jazz, Jazz, Funk, Jazz, Jazz, Jazz, Jazz, Jazz, Soul, Jazz]      1
[Folk, Rock, Rock, Folk]                                          1
[Rock, Pop, Pop, Rock, Pop, Rock]                                 1
Name: count, Length: 1541, dtype: int64


In [127]:
import pandas as pd
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("asaniczka/top-spotify-songs-in-73-countries-daily-updated")

# Load the dataset
csv_path = f"{path}/universal_top_spotify_songs.csv"
spotify_df = pd.read_csv(csv_path)

# Step 1: Clean column names just to be sure (remove leading/trailing spaces)
spotify_df.columns = spotify_df.columns.str.strip()
audio_features_df.columns = audio_features_df.columns.str.strip()

# Step 2: Build mapping with one country per track
track_country_df = spotify_df[['spotify_id', 'country']].dropna().drop_duplicates(subset='spotify_id')

# Step 3: Merge correctly
audio_features_with_country = pd.merge(
    audio_features_df,
    track_country_df,
    how='left',
    left_on='spotify_track_id',
    right_on='spotify_id'
)

# Step 4: Clean up and check results
audio_features_with_country.drop(columns=['spotify_id'], inplace=True, errors='ignore')

# ✅ Final verification
print(audio_features_with_country[['spotify_track_id', 'country']].head())

audio_features_with_country.to_csv("broad_genres_country.csv", index=False)


         spotify_track_id country
0                     NaN     NaN
1  3tvqPPpXyIgKrm4PR9HCf0     NaN
2  1fHHq3qHU8wpRKHzhojZ4a     NaN
3  3MjUtNVVq3C8Fn0MP3zhXa     NaN
4  2yLa0QULdQr0qAIvVwN6B5      IE


In [128]:
# Drop NaN values first to avoid counting them
unique_countries = audio_features_with_country['country'].dropna().unique()

# Count
num_unique_countries = len(unique_countries)

# Display results
print(f"🌍 Number of unique countries: {num_unique_countries}")
print("📄 Country codes:")
print(sorted(unique_countries))


🌍 Number of unique countries: 43
📄 Country codes:
['AE', 'AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CR', 'CZ', 'DE', 'EE', 'FR', 'GB', 'HK', 'HU', 'ID', 'IE', 'KR', 'KZ', 'LT', 'LU', 'LV', 'MY', 'NI', 'NL', 'NO', 'NZ', 'PH', 'PL', 'PT', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TW', 'UA', 'US', 'VE', 'ZA']


In [129]:
import pycountry

# Function to convert ISO country codes to full names
def code_to_name(code):
    try:
        return pycountry.countries.get(alpha_2=code).name
    except:
        return code  # If code is invalid or missing

# Apply the function to create a new column
audio_features_with_country['country_name'] = audio_features_with_country['country'].apply(code_to_name)

# ✅ Preview result
print(audio_features_with_country[['country', 'country_name']].dropna().head())


    country               country_name
4        IE                    Ireland
35       US              United States
133      NZ                New Zealand
277      TW  Taiwan, Province of China
307      BG                   Bulgaria


In [130]:
from collections import Counter

# Function to get the most frequent genre in a list
def most_frequent_genre(genre_list):
    if not isinstance(genre_list, list) or len(genre_list) == 0:
        return None
    counter = Counter(genre_list)
    return counter.most_common(1)[0][0]

# Apply the function to create the new column
audio_features_with_country['top_genre'] = audio_features_with_country['broad_genre'].apply(most_frequent_genre)

# Preview result
print(audio_features_with_country[['broad_genre', 'top_genre']].head())


                                         broad_genre       top_genre
0                                              [Pop]             Pop
1  [Easy Listening, Pop, Easy Listening, Easy Lis...  Easy Listening
2                                  [Rock, Pop, Rock]            Rock
3                                    [Pop, Pop, Pop]             Pop
4                                         [Pop, Pop]             Pop


In [131]:
# Flatten all lists in 'broad_genre' into one list
all_broad_genres = [
    genre
    for genre_list in audio_features_with_country['broad_genre']
    if isinstance(genre_list, list)
    for genre in genre_list
]

# Get unique genres
unique_broad_genres = sorted(set(all_broad_genres))

# Print results
print(f"🎵 Found {len(unique_broad_genres)} unique broad genres:")
print(unique_broad_genres)


🎵 Found 24 unique broad genres:
['Blues', 'Classical', 'Comedy', 'Country', 'Dance', 'Easy Listening', 'Electronic', 'Folk', 'Funk', 'Hip Hop', 'Holiday', 'Indie', 'Jazz', 'Latin', 'Metal', 'Other', 'Pop', 'R&B', 'Reggae', 'Regional Mexican', 'Rock', 'Ska', 'Soul', 'Soundtrack']


In [132]:
# Lowercase each genre in the list inside 'broad_genre'
audio_features_with_country['broad_genre'] = audio_features_with_country['broad_genre'].apply(
    lambda genres: [g.lower() for g in genres] if isinstance(genres, list) else genres
)

# Lowercase the string in 'top_genre'
audio_features_with_country['top_genre'] = audio_features_with_country['top_genre'].apply(
    lambda g: g.lower() if isinstance(g, str) else g
)

# Preview result
print(audio_features_with_country[['broad_genre', 'top_genre']].head())
audio_features_with_country.to_csv("broad_genres_country.csv", index=False)


                                         broad_genre       top_genre
0                                              [pop]             pop
1  [easy listening, pop, easy listening, easy lis...  easy listening
2                                  [rock, pop, rock]            rock
3                                    [pop, pop, pop]             pop
4                                         [pop, pop]             pop


In [133]:
import pandas as pd

# Step 1: List the numeric audio features available
features = ['danceability', 'energy', 'valence', 'tempo', 'popularity']

# Step 2: Drop rows with missing country
spotify_df_filtered = spotify_df.dropna(subset=['country'])

# Step 3: Compute the mean of each feature per country
country_feature_means = spotify_df_filtered.groupby('country')[features].mean().reset_index()

# Step 4: Preview result
print(" Mean audio features per country:")
print(country_feature_means.head())


 Mean audio features per country:
  country  danceability    energy   valence       tempo  popularity
0      AE      0.640262  0.624708  0.496965  122.196612   86.407499
1      AR      0.698864  0.684404  0.638535  117.079778   77.952501
2      AT      0.666465  0.659294  0.529321  127.303589   81.442627
3      AU      0.630392  0.623838  0.512097  125.577155   87.415555
4      BE      0.657899  0.650700  0.524811  125.329865   83.745316


In [134]:

# Drop NaN values first to avoid counting them
unique_countries = country_feature_means['country'].dropna().unique()

# Count
num_unique_countries = len(unique_countries)

# Display results
print(f"🌍 Number of unique countries: {num_unique_countries}")
print("📄 Country codes:")
print(sorted(unique_countries))


🌍 Number of unique countries: 72
📄 Country codes:
['AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CZ', 'DE', 'DK', 'DO', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JP', 'KR', 'KZ', 'LT', 'LU', 'LV', 'MA', 'MX', 'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'PA', 'PE', 'PH', 'PK', 'PL', 'PT', 'PY', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TR', 'TW', 'UA', 'US', 'UY', 'VE', 'VN', 'ZA']


In [135]:
# Filter rows where 'country_name' is not NaN
country_filtered_df = audio_features_with_country.dropna(subset=['country_name'])



In [136]:
# Step 1: Add 'avg_' prefix to the mean feature columns (except 'country')
renamed_means = country_feature_means.rename(
    columns={col: f'avg_{col}' for col in country_feature_means.columns if col != 'country'}
)

# Step 2: Merge on the 'country' column
merged_df = pd.merge(
    country_filtered_df,
    renamed_means,
    how='left',
    on='country'
)

merged_df.to_csv("country_genre_data_viz.csv", index=False)
